In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy
import gensim
import gensim.corpora as corpora
from gensim.models import LdaModel
from IPython.display import display, Markdown  # Para exibir Markdown no Colab
import os  # Para criar diretórios
from google.colab import drive  # Importa a API do Google Drive

# --- Configurações Iniciais ---

# 1. Montar o Google Drive
drive.mount('/content/drive')

# 2. Caminho base no Google Drive
drive_base_path = '/content/drive/MyDrive/'

# 3. Criação do diretório para salvar os gráficos no Drive
pasta_graficos = "graficos"
drive_graficos_path = os.path.join(drive_base_path, pasta_graficos)
if not os.path.exists(drive_graficos_path):
    os.makedirs(drive_graficos_path)

# 4. URL do dataset (Google Sheets)
url_dataset = "https://docs.google.com/spreadsheets/d/1xI-kcwIx0aFQAi5JxsjuAknPiBmLlCW0dzE3ag3s0Io/export?format=csv"

# --- Configurações de Estilo dos Gráficos ---

sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 20,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
})
DEFAULT_PALETTE = "viridis"
FIGSIZE = (12, 8)

# --- Funções Auxiliares ---

def create_figure():
    """Cria uma figura com as configurações padrão."""
    return plt.figure(figsize=FIGSIZE)

def save_fig(fig, filename):
    """Salva a figura no Drive."""
    filepath = os.path.join(drive_graficos_path, filename)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo: {filepath}")
    plt.close(fig)  # Importante para liberar memória

# --- Funções de Análise ---

def carregar_dados(url):
    """Carrega os dados do Google Sheets (CSV) em um DataFrame Pandas."""
    try:
        df = pd.read_csv(url)
        print("Dados carregados com sucesso!")
        return df
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None

def pre_processar_dados(df):
    """
    Realiza o pré-processamento inicial dos dados (limpeza, tratamento de missings, etc.).
    """
    print("Pré-processamento dos dados...")
    # Converte nomes de colunas para minúsculas e substitui espaços por _
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

    # Exemplo de tratamento de valores ausentes (preencher com a média, se numérico)
    for col in df.columns:
        if df[col].isnull().any():
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(df[col].mean())
                print(f"Valores ausentes na coluna '{col}' preenchidos com a média.")
            else:
                # Se não for numérico, você pode preencher com a moda, um valor constante, ou dropar as linhas
                df[col] = df[col].fillna(df[col].mode()[0])  # Ex: preencher com a moda
                print(f"Valores ausentes na coluna '{col}' preenchidos com a moda.")
    return df



def analisar_sentimento(df, coluna_texto):
    """
    Analisa o sentimento de uma coluna de texto do DataFrame.
    Retorna o DataFrame com uma nova coluna 'sentimento'.
    """
    print("Analisando sentimento...")
    nltk.download('vader_lexicon', quiet=True)
    sia = SentimentIntensityAnalyzer()

    def get_sentiment_score(text):
        """Calcula o score de sentimento composto (VADER)."""
        if isinstance(text, str):
            return sia.polarity_scores(text)['compound']
        else:
            return 0.0

    df['sentimento'] = df[coluna_texto].apply(get_sentiment_score)
    return df

def visualizar_distribuicao_sentimento(df, coluna_sentimento='sentimento'):
    """
    Visualiza a distribuição dos scores de sentimento usando um histograma.
    """
    fig = create_figure()
    sns.histplot(data=df, x=coluna_sentimento, kde=True, color='cyan')  # Cor consistente
    plt.title('Distribuição dos Scores de Sentimento')
    plt.xlabel('Score de Sentimento (VADER)')
    plt.ylabel('Frequência')
    save_fig(fig, 'distribuicao_sentimento.png')



def modelagem_de_topicos(df, coluna_texto, num_topicos=5):
    """
    Realiza a modelagem de tópicos (LDA).
    """
    print("Realizando modelagem de tópicos (LDA)...")

    try:
        nlp = spacy.load("pt_core_news_sm")
    except:
        print("Modelo 'pt_core_news_sm' não encontrado. Tentando baixar...")
        try:
            spacy.cli.download("pt_core_news_sm")
            nlp = spacy.load("pt_core_news_sm")
        except:
            print("Não foi possível baixar o modelo. Verifique sua conexão.")
            return None, None

    def preprocessar_texto(texto):
        if isinstance(texto, str):
            doc = nlp(texto)
            tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
            return tokens
        else:
            return []

    textos_processados = df[coluna_texto].apply(preprocessar_texto)
    id2word = corpora.Dictionary(textos_processados)
    corpus = [id2word.doc2bow(texto) for texto in textos_processados]

    lda_model = LdaModel(corpus=corpus,
                         id2word=id2word,
                         num_topics=num_topicos,
                         random_state=42,
                         passes=10)

    for idx, topic in lda_model.print_topics(-1):
        print(f"Tópico: {idx} \nPalavras: {topic}\n")

    return lda_model, corpus



def visualizar_topicos(lda_model, corpus, num_palavras=10):
    """
    Visualiza os tópicos gerados pelo modelo LDA (gráfico de barras).
    """
    topicos = lda_model.show_topics(num_topics=-1, num_words=num_palavras, formatted=False)

    for topico_num, topico in topicos:
        palavras, pesos = zip(*topico)
        fig = create_figure()
        plt.barh(palavras, pesos, color='cyan')  # Cor consistente
        plt.title(f'Tópico {topico_num}')
        plt.xlabel('Peso')
        plt.ylabel('Palavras')
        plt.gca().invert_yaxis()
        save_fig(fig, f'topico_{topico_num}.png')


def analisar_correlacoes(df, colunas_numericas):
    """
    Analisa correlações entre colunas numéricas.
    """
    print("Analisando correlações...")
    if len(colunas_numericas) < 2:
        print("Não há colunas numéricas suficientes.")
        return None

    try:
        correlacao = df[colunas_numericas].corr()
        print("\nMatriz de Correlação:\n", correlacao)

        fig = create_figure()
        sns.heatmap(correlacao, annot=True, cmap=DEFAULT_PALETTE, fmt=".2f",
                    cbar_kws={'label': 'Correlação'})  # Adiciona legenda
        plt.title('Matriz de Correlação')
        save_fig(fig, 'matriz_correlacao.png')
        return correlacao

    except Exception as e:
        print(f"Erro na análise de correlação: {e}")
        return None


# --- Fluxo Principal (Main) ---

if __name__ == "__main__":
    display(Markdown("# Análise das Representações da IA no Cinema"))

    # 1. Carregar os dados
    df = carregar_dados(url_dataset)
    if df is None:
        exit()

    # 2. Pré-processar os dados
    df = pre_processar_dados(df)

    # 3. Mostrar informações básicas
    display(Markdown("## Dados Carregados"))
    print(df.head())
    print(df.info())
    print(df.describe())

    # 4. Análise de Sentimento
    colunas_texto = df.select_dtypes(include='object').columns
    if len(colunas_texto) > 0:
        df = analisar_sentimento(df, colunas_texto[0])
        visualizar_distribuicao_sentimento(df)
    else:
        print("Nenhuma coluna de texto encontrada para análise de sentimento.")

    # 5. Modelagem de Tópicos
    if len(colunas_texto) > 0:
        lda_model, corpus = modelagem_de_topicos(df, colunas_texto[0])
        if lda_model is not None:
            visualizar_topicos(lda_model, corpus)
    else:
        print("Nenhuma coluna de texto encontrada para modelagem de tópicos.")

    # 6. Análise de Correlações
    colunas_numericas = df.select_dtypes(include=np.number).columns.tolist()
    analisar_correlacoes(df, colunas_numericas)

    display(Markdown("## Conclusões Preliminares"))
    display(Markdown("Este notebook realizou análises iniciais. Próximas etapas: refinamento do pré-processamento, análise de outras colunas, implementação da análise visual, e integração com outras técnicas (Scikit-learn, XGBoost, SHAP, etc.)."))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Análise das Representações da IA no Cinema

Dados carregados com sucesso!
Pré-processamento dos dados...


## Dados Carregados

   filme_id                  titulo   ano             genero  \
0         1  A Revolta das Máquinas  2023  Ficção Científica   
1         2        O Código da Vida  2022              Drama   
2         3      O Androide Perdido  2021  Ficção Científica   
3         4      Ameaça Cibernética  2020           Suspense   
4         5           O Sonho da IA  2019              Drama   

                                             dialogo  bilheteria  \
0  “As máquinas estão se rebelando! Precisamos de...   120000000   
1  “A inteligência artificial pode curar doenças,...    85000000   
2  “Eu sou um androide, mas também tenho sentimen...    60000000   
3  “O hacker invadiu o sistema e agora controla t...    95000000   
4  “Eu criei uma IA que pode compor músicas incrí...    70000000   

  avaliacao_critica  
0             8.5 |  
1             7.9 |  
2             6.8 |  
3             8.2 |  
4             7.5 |  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Tópico: 0 
Palavras: 0.215*"ia" + 0.112*"futuro" + 0.061*"enigma" + 0.061*"androide" + 0.061*"perdido" + 0.061*"criação" + 0.010*"máquina" + 0.010*"vida" + 0.010*"máquinas" + 0.010*"artificial"

Tópico: 1 
Palavras: 0.068*"artificial" + 0.068*"digital" + 0.068*"cibernética" + 0.068*"inteligência" + 0.068*"ameaça" + 0.068*"cérebro" + 0.068*"legado" + 0.068*"turing" + 0.012*"ia" + 0.012*"futuro"

Tópico: 2 
Palavras: 0.124*"máquinas" + 0.069*"vida" + 0.068*"criação" + 0.068*"algoritmo" + 0.068*"limite" + 0.068*"revolta" + 0.068*"visão" + 0.012*"ia" + 0.011*"máquina" + 0.011*"artificial"

Tópico: 3 
Palavras: 0.353*"ia" + 0.062*"máquina" + 0.023*"potencial" + 0.023*"dilema" + 0.023*"propósito" + 0.023*"sonho" + 0.023*"coração" + 0.023*"amanhecer" + 0.023*"mistério" + 0.023*"despertar"

Tópico: 4 
Palavras: 0.111*"ia" + 0.061*"promessa" + 0.061*"robôs" + 0.061*"esperança" + 0.061*"rede" + 0.061*"consciência" + 0.061*"neural" + 0.061*"mundo" + 0.061*"artificial" + 0.011*"máquinas"

Gráfico 

## Conclusões Preliminares

Este notebook realizou análises iniciais. Próximas etapas: refinamento do pré-processamento, análise de outras colunas, implementação da análise visual, e integração com outras técnicas (Scikit-learn, XGBoost, SHAP, etc.).